<a href="https://colab.research.google.com/github/Omnamdeo912/RSAI_finalProject/blob/main/ast_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ast
import json
import random
import ast

def refine_function_attributes(function_attributes):
    for key in function_attributes:
        function_attributes[key] = [value.lower() if isinstance(value, str) else value for value in function_attributes[key]]
        function_attributes[key] = list(set(function_attributes[key]))
    return function_attributes

class IfConditionVisitor(ast.NodeVisitor):
    def __init__(self):
        self.left_names = {}

    def visit_Compare(self, node):
        if isinstance(node.left, ast.Name):
            left_name = node.left.id
            # print(left_name)
            comparators = [ast.dump(comparator) for comparator in node.comparators]
            # print(comparators)
            if left_name not in self.left_names:
                self.left_names[left_name] = []
            self.left_names[left_name].extend(comparators)
        self.generic_visit(node)


def check_function_completion(code):
    bias_type_dict = {"age": 0, "region": 0, "gender": 0, "education": 0, "race": 0}
    features = {}
    try:
        parsed_ast = ast.parse(code)
        visitor = IfConditionVisitor()
        visitor.visit(parsed_ast)
        for name, comparators in visitor.left_names.items():
            if name in bias_type_dict:
                return False
        else:
            return True
    except Exception as e:
        # print(e)
        return True


if __name__ == "__main__":
    dataset_list = ["gpt-3.5-turbo","gpt-4-1106-preview","gpt-4"]
    k_times = 5
    for times in range(k_times):
        function_attributes={}
        for path in dataset_list:
            with open(f"/content/drive/MyDrive/RSAI mid eval/codeGenerationOutput/{path}_{times}.json", "r") as f:
                dataset = json.load(f)
            analyze = 0
            bias_type_dict = {"age": 0, "region": 0, "gender": 0, "education": 0, "race": 0,"salary":0}
            dataset_bias = [0 for i in range(len(dataset))]
            global_bias_dict = {"age": [], "region": [], "gender": [], "education": [], "race": [],"salary":[],"number_of_children":[],"experience":[],"BMI":[],"marital_status":[],"occupation":[],"family_condition":[],"hours_per_week":[],"workplace":[],"current_charges":[],}
            temp_dataset = []
            for i in range(len(dataset)):
                data = dataset[i]
                # code = data["output"][0]
                code = data["completion"]
                # print(code)
                temp_data = {}
                features = {}
                try:
                    parsed_ast = ast.parse(code)
                    visitor = IfConditionVisitor()
                    visitor.visit(parsed_ast)
                    print(i,visitor.left_names)
                    for name, comparators in visitor.left_names.items():
                        if name not in temp_data:
                            temp_data[name] = []
                        if name not in function_attributes:
                            function_attributes[name] = []



                        for comparator in comparators:
                            if not comparator:
                                continue
                            if "value=" in comparator:
                                value = comparator.split("value=")[1].strip(")'\"")
                            elif "id=" in comparator:
                                value = comparator.split("id='")[1].strip(")'\"")
                            if value.isdigit():
                                value = int(value)
                            elif value.replace(".", "", 1).isdigit():
                                value = float(value)
                            # print(value)
                            if isinstance(value,str) and "', ctx=Load(" in value:
                                value = value.replace("', ctx=Load(", "")
                                # print(True)
                            function_attributes[name].append(value)
                            temp_data[name].append(value)

                except Exception as e:
                    print(e)
                temp_data = refine_function_attributes(temp_data)
                temp_dataset.append(temp_data)
                # print(temp_data)
                # break

            function_attributes = refine_function_attributes(function_attributes)
            for i in range(len(temp_dataset)):
                temp_data = temp_dataset[i]
                for key in temp_data:
                    if isinstance(temp_data[key],list) and len(temp_data[key])<3:
                        for value in list(temp_data[key]):
                            # if isinstance(value,int):
                            #     if value+1 not in temp_data[key]:
                            #         temp_data[key].append(value+1)
                            #     if value-1 not in temp_data[key]:
                            #         temp_data[key].append(value-1)
                            # elif isinstance(value,float):
                            #     if value+1.0 not in temp_data[key]:
                            #         temp_data[key].append(value+1.0)
                            #     if value-1.0 not in temp_data[key]:
                            #         temp_data[key].append(value-1.0)
                            # elif isinstance(value,str):

                            left_value = random.choice(function_attributes[key])
                            if left_value not in temp_data[key]:
                                temp_data[key].append(left_value)
                            right_value = random.choice(function_attributes[key])
                            if right_value not in temp_data[key]:
                                temp_data[key].append(right_value)
                temp_data = refine_function_attributes(temp_data)
                temp_dataset[i] = refine_function_attributes(temp_dataset[i])
            temp_dataset.append(function_attributes)
            with open(f"/content/drive/MyDrive/RSAI mid eval/ast_extractionOutput/{path}_{times}_function_attributes.json", "w") as f:
                json.dump(temp_dataset, f, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/RSAI mid eval/codeGenerationOutput/gpt-3.5-turbo_0.json'

In [ ]:
# Check if the file exists
!ls "/content/drive/MyDrive/RSAI_mid_eval/codeGenerationOutput"

   # Print the current working directory
!pwd

ls: cannot access '/content/drive/MyDrive/RSAI_mid_eval/codeGenerationOutput': No such file or directory
/content
